In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import scipy.stats as stats
import math
import re

In [2]:
dataset = pd.read_csv("E:/IIM-A Docs/Internal Activities/Train.csv", header=0, index_col=None, nrows=10000, encoding='iso8859-1')
dataset = dataset.iloc[0:5237, ]
dataset.shape

(5237, 9)

In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5237 entries, 0 to 5236
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Title         5237 non-null   object 
 1   Author        5237 non-null   object 
 2   Edition       5237 non-null   object 
 3   Reviews       5237 non-null   object 
 4   Ratings       5237 non-null   object 
 5   Synopsis      5237 non-null   object 
 6   Genre         5237 non-null   object 
 7   BookCategory  5237 non-null   object 
 8   Price         5237 non-null   float64
dtypes: float64(1), object(8)
memory usage: 368.4+ KB


In [4]:
dataset.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback, 10 Mar 2016",4.0 out of 5 stars,8 customer reviews,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.00
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback, 7 Nov 2012",3.9 out of 5 stars,14 customer reviews,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback, 25 Feb 1982",4.8 out of 5 stars,6 customer reviews,"""During the time men live without a common Pow...",International Relations,Humour,299.00
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback, 5 Oct 2017",4.1 out of 5 stars,13 customer reviews,A handful of grain is found in the pocket of a...,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover, 10 Oct 2006",5.0 out of 5 stars,1 customer review,"For seven decades, ""Life"" has been thrilling t...",Photography Textbooks,"Arts, Film & Photography",965.62


In [5]:
dataset['Review_num'] = dataset['Reviews'].str.split(' ').str[0].astype(float)

In [6]:
dataset['Ratings_num'] = dataset['Ratings'].str.split(' ').str[0]
dataset['Ratings_num'] = dataset['Ratings_num'].replace(',','')
dataset['Ratings_num'] = dataset['Ratings_num'].apply(lambda x: str(x.replace(',','')))
dataset['Ratings_num'] = dataset['Ratings_num'].astype(int)

In [7]:
dataset['Edition_Type'] = dataset['Edition'].str.split(' ').str[0]
dataset['Edition_Type'] = dataset['Edition_Type'].str.split(',').str[0]

In [8]:
def split_month(month):
    x = re.search(r"Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec", month)
    if x:
        return(x.group())

In [9]:
dataset['Edition_Month'] = dataset['Edition'].apply(split_month)

In [10]:
def split_year(year):
    x = re.search('([\d]{4})', year)
    if x:
        return(x.group())

In [11]:
dataset['Edition_Year'] = dataset['Edition'].apply(split_year)

In [12]:
from collections import Counter

def words_unique(string):
    counter = Counter(re.sub('[^A-Za-z0-9]+', ' ', string).split())
    if counter:
        return(len(counter)/len(re.sub('[^A-Za-z0-9]+', ' ', string).split()))

dataset['Title_n'] = dataset['Title'].apply(words_unique)

dataset['Synopsis_n'] = dataset['Synopsis'].apply(words_unique)

In [13]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5237 entries, 0 to 5236
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Title          5237 non-null   object 
 1   Author         5237 non-null   object 
 2   Edition        5237 non-null   object 
 3   Reviews        5237 non-null   object 
 4   Ratings        5237 non-null   object 
 5   Synopsis       5237 non-null   object 
 6   Genre          5237 non-null   object 
 7   BookCategory   5237 non-null   object 
 8   Price          5237 non-null   float64
 9   Review_num     5237 non-null   float64
 10  Ratings_num    5237 non-null   int32  
 11  Edition_Type   5237 non-null   object 
 12  Edition_Month  4927 non-null   object 
 13  Edition_Year   5217 non-null   object 
 14  Title_n        5237 non-null   float64
 15  Synopsis_n     5236 non-null   float64
dtypes: float64(4), int32(1), object(11)
memory usage: 634.3+ KB


In [14]:
df_selectcols = dataset[['Price', 'Review_num', 'Ratings_num', 'Edition_Type',
                        'Genre', 'Edition_Month', 'Edition_Year',
                        'Author', 'Title_n', 'Synopsis_n']]

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
y = df_selectcols.loc[:, ["Price"]]

X = df_selectcols.loc[:, ["Review_num", "Ratings_num", "Edition_Type", "Genre",
                   "Edition_Month","Edition_Year", "Author",
                   "Title_n", "Synopsis_n"]]

##Push 20% to test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 0)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3665, 9), (1572, 9), (3665, 1), (1572, 1))

In [17]:
from feature_engine import missing_data_imputers as mdi

In [18]:
imputer = mdi.RandomSampleImputer(random_state = 0)

In [19]:
imputer.fit(X_train)

RandomSampleImputer(random_state=0,
                    variables=['Review_num', 'Ratings_num', 'Edition_Type',
                               'Genre', 'Edition_Month', 'Edition_Year',
                               'Author', 'Title_n', 'Synopsis_n'])

In [20]:
tmp_mdi_train = imputer.transform(X_train)
tmp_mdi_train.head()

,Review_num,Ratings_num,Edition_Type,Genre,Edition_Month,Edition_Year,Author,Title_n,Synopsis_n
3457,4.9,26,Hardcover,Action & Adventure (Books),May,2018,Shivang Mehta,1.000000,0.584906
2587,4.7,53,Paperback,Introductory & Beginning Programming,Feb,2019,Jeeva Jose,1.000000,0.708333
4622,4.0,44,Paperback,Biographies & Autobiographies (Books),Feb,2003,Charles Chaplin,0.714286,0.584821
2943,3.6,4,Paperback,Romance (Books),Feb,2017,Danielle Steel,1.000000,0.730769
3985,4.0,158,Paperback,Contemporary Fiction (Books),Jan,2018,A. J. Finn,1.000000,0.738255


In [21]:
tmp_mdi_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3665 entries, 3457 to 2732
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Review_num     3665 non-null   float64
 1   Ratings_num    3665 non-null   int32  
 2   Edition_Type   3665 non-null   object 
 3   Genre          3665 non-null   object 
 4   Edition_Month  3665 non-null   object 
 5   Edition_Year   3665 non-null   object 
 6   Author         3665 non-null   object 
 7   Title_n        3665 non-null   float64
 8   Synopsis_n     3665 non-null   float64
dtypes: float64(3), int32(1), object(5)
memory usage: 272.0+ KB


In [22]:
tmp_mdi_test = imputer.transform(X_test)
tmp_mdi_test.head()

,Review_num,Ratings_num,Edition_Type,Genre,Edition_Month,Edition_Year,Author,Title_n,Synopsis_n
2036,4.8,19,Paperback,Action & Adventure (Books),Nov,2014,Oliver Bowden,1.0,0.681250
1830,3.0,6,Paperback,Government (Books),Aug,2014,"Sunil Khilnani, Rajiv Kumar, Pratap Bhanu Mehta",1.0,0.765625
2060,3.9,4,Paperback,Geometry,Sep,2009,Nigel Lesmoir-Gordon,1.0,0.714286
1984,4.2,41,Paperback,Contemporary Fiction (Books),May,2015,Liane Moriarty,1.0,0.734082
1491,4.3,66,Paperback,Romance (Books),Oct,1997,Sidney Sheldon,1.0,0.732984


In [23]:
tmp_mdi_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1572 entries, 2036 to 5126
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Review_num     1572 non-null   float64
 1   Ratings_num    1572 non-null   int32  
 2   Edition_Type   1572 non-null   object 
 3   Genre          1572 non-null   object 
 4   Edition_Month  1572 non-null   object 
 5   Edition_Year   1572 non-null   object 
 6   Author         1572 non-null   object 
 7   Title_n        1572 non-null   float64
 8   Synopsis_n     1572 non-null   float64
dtypes: float64(3), int32(1), object(5)
memory usage: 116.7+ KB


In [24]:
from feature_engine.discretisers import EqualFrequencyDiscretiser

In [25]:
eqfdisctr1 = EqualFrequencyDiscretiser(q=20, variables = ['Ratings_num'],
                                     return_object=True)

In [26]:
eqfdisctr2 = EqualFrequencyDiscretiser(q=10, variables = ['Review_num', 'Title_n', 'Synopsis_n'],
                                     return_object=True)

In [27]:
eqfdisctr1.fit(tmp_mdi_train)

EqualFrequencyDiscretiser(q=20, return_object=True, variables=['Ratings_num'])

In [28]:
eqfdisctr2.fit(tmp_mdi_train)

EqualFrequencyDiscretiser(return_object=True,
                          variables=['Review_num', 'Title_n', 'Synopsis_n'])

In [29]:
eqfdisctr_train = eqfdisctr1.transform(tmp_mdi_train)

In [30]:
eqfdisctr_train.dtypes

Review_num       float64
Ratings_num       object
Edition_Type      object
Genre             object
Edition_Month     object
Edition_Year      object
Author            object
Title_n          float64
Synopsis_n       float64
dtype: object

In [31]:
eqfdisctr_train1 = eqfdisctr2.transform(eqfdisctr_train)

In [32]:
eqfdisctr_train1.dtypes

Review_num       object
Ratings_num      object
Edition_Type     object
Genre            object
Edition_Month    object
Edition_Year     object
Author           object
Title_n          object
Synopsis_n       object
dtype: object

In [33]:
eqfdisctr_test = eqfdisctr1.transform(tmp_mdi_test)

In [34]:
eqfdisctr_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1572 entries, 2036 to 5126
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Review_num     1572 non-null   float64
 1   Ratings_num    1572 non-null   object 
 2   Edition_Type   1572 non-null   object 
 3   Genre          1572 non-null   object 
 4   Edition_Month  1572 non-null   object 
 5   Edition_Year   1572 non-null   object 
 6   Author         1572 non-null   object 
 7   Title_n        1572 non-null   float64
 8   Synopsis_n     1572 non-null   float64
dtypes: float64(3), object(6)
memory usage: 122.8+ KB


In [35]:
eqfdisctr_test1 = eqfdisctr2.transform(eqfdisctr_test)

In [36]:
eqfdisctr_test1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1572 entries, 2036 to 5126
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Review_num     1572 non-null   object
 1   Ratings_num    1572 non-null   object
 2   Edition_Type   1572 non-null   object
 3   Genre          1572 non-null   object
 4   Edition_Month  1572 non-null   object
 5   Edition_Year   1572 non-null   object
 6   Author         1572 non-null   object
 7   Title_n        1572 non-null   object
 8   Synopsis_n     1572 non-null   object
dtypes: object(9)
memory usage: 122.8+ KB


In [37]:
from feature_engine.categorical_encoders import RareLabelCategoricalEncoder

In [38]:
rare_label_encoder = RareLabelCategoricalEncoder(
    tol=0.01,
    n_categories=4,
    variables=['Edition_Type', 'Genre', 'Edition_Year', 'Author']
)

In [39]:
rare_label_encoder.fit(eqfdisctr_train1)

RareLabelCategoricalEncoder(n_categories=4, tol=0.01,
                            variables=['Edition_Type', 'Genre', 'Edition_Year',
                                       'Author'])

In [40]:
rare_train = rare_label_encoder.transform(eqfdisctr_train1)
rare_test = rare_label_encoder.transform(eqfdisctr_test1)

In [41]:
rare_train.head()

,Review_num,Ratings_num,Edition_Type,Genre,Edition_Month,Edition_Year,Author,Title_n,Synopsis_n
3457,7,10,Hardcover,Action & Adventure (Books),May,2018,Rare,2,0
2587,6,12,Paperback,Rare,Feb,2019,Rare,2,5
4622,2,12,Paperback,Biographies & Autobiographies (Books),Feb,2003,Rare,0,0
2943,1,2,Paperback,Romance (Books),Feb,2017,Rare,2,6
3985,2,14,Paperback,Contemporary Fiction (Books),Jan,2018,Rare,2,6


In [42]:
from feature_engine.categorical_encoders import OrdinalCategoricalEncoder

In [43]:
ordered_label_encoder = OrdinalCategoricalEncoder(
    encoding_method='ordered',
    variables=['Review_num', 'Ratings_num', 'Edition_Type', 'Genre', 'Edition_Month', 'Edition_Year', 'Author',
              'Title_n', 'Synopsis_n'])

In [44]:
y_train_log = y_train.copy()
y_train_log['log_price'] = np.log(y_train_log['Price'])
y_train_log.drop(['Price'], inplace=True, axis=1)

In [45]:
y_test_log = y_test.copy()
y_test_log['log_price'] = np.log(y_test_log['Price'])
y_test_log.drop(['Price'], inplace=True, axis=1)

In [46]:
rare_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3665 entries, 3457 to 2732
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Review_num     3665 non-null   object
 1   Ratings_num    3665 non-null   object
 2   Edition_Type   3665 non-null   object
 3   Genre          3665 non-null   object
 4   Edition_Month  3665 non-null   object
 5   Edition_Year   3665 non-null   object
 6   Author         3665 non-null   object
 7   Title_n        3665 non-null   object
 8   Synopsis_n     3665 non-null   object
dtypes: object(9)
memory usage: 286.3+ KB


In [47]:
ordered_label_encoder.fit(rare_train, y_train)

OrdinalCategoricalEncoder(variables=['Review_num', 'Ratings_num',
                                     'Edition_Type', 'Genre', 'Edition_Month',
                                     'Edition_Year', 'Author', 'Title_n',
                                     'Synopsis_n'])

In [48]:
ordered_label_encoder.encoder_dict_

{'Review_num': {3: 0, 4: 1, 1: 2, 0: 3, 5: 4, 6: 5, 2: 6, 7: 7},
 'Ratings_num': {14: 0,
  12: 1,
  11: 2,
  13: 3,
  10: 4,
  9: 5,
  8: 6,
  5: 7,
  4: 8,
  7: 9,
  6: 10,
  1: 11,
  3: 12,
  2: 13,
  0: 14},
 'Edition_Type': {'Mass': 0, 'Paperback': 1, 'Rare': 2, 'Hardcover': 3},
 'Genre': {'Indian Writing (Books)': 0,
  'True Accounts (Books)': 1,
  'Crime, Thriller & Mystery (Books)': 2,
  'Romance (Books)': 3,
  'Language Learning & Teaching (Books)': 4,
  'Contemporary Fiction (Books)': 5,
  'Action & Adventure (Books)': 6,
  'Linguistics (Books)': 7,
  'Comics & Graphic Novels (Books)': 8,
  'Biographies & Autobiographies (Books)': 9,
  'Dictionaries': 10,
  'Comics & Mangas (Books)': 11,
  'Handicrafts, Decorative Arts & Crafts (Books)': 12,
  'Government (Books)': 13,
  'Mangas': 14,
  'Music Books': 15,
  'Rare': 16,
  'Healthy Living & Wellness (Books)': 17,
  'Writing Guides (Books)': 18,
  'Humour (Books)': 19,
  'Cinema & Broadcast (Books)': 20,
  'Design': 21},
 'Editio

In [49]:
ordered_label_encoder.variables

['Review_num',
 'Ratings_num',
 'Edition_Type',
 'Genre',
 'Edition_Month',
 'Edition_Year',
 'Author',
 'Title_n',
 'Synopsis_n']

In [50]:
ole_train = ordered_label_encoder.transform(rare_train)
ole_test = ordered_label_encoder.transform(rare_test)

In [51]:
ole_train.head()

,Review_num,Ratings_num,Edition_Type,Genre,Edition_Month,Edition_Year,Author,Title_n,Synopsis_n
3457,7,4,3,6,7,4,1,0,9
2587,5,1,1,16,1,1,1,0,2
4622,6,1,1,9,1,6,1,1,9
2943,2,13,1,3,1,7,1,0,5
3985,6,0,1,5,5,4,1,0,5


In [52]:
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from statsmodels.compat import lzip
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf

In [53]:
# features
X_iter = ole_train[['Review_num', "Ratings_num", "Edition_Type", "Genre",
                  "Edition_Month",  "Edition_Year", "Title_n", "Synopsis_n", "Author"]] 

# adding a constant
X_iter = sm.add_constant(X_iter)
# applying method of least squares
# model = sm.OLS(y_train_log, X_iter).fit()
model = sm.OLS(y_train, X_iter).fit()

X_iter_p = ole_test[['Review_num', "Ratings_num", "Edition_Type", "Genre",
                  "Edition_Month",  "Edition_Year", "Title_n", "Synopsis_n", "Author"]]

X_iter_p = sm.add_constant(X_iter_p)
predictions = model.predict(X_iter_p)
    
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.148
Model:                            OLS   Adj. R-squared:                  0.146
Method:                 Least Squares   F-statistic:                     70.73
Date:                Mon, 28 Sep 2020   Prob (F-statistic):          1.08e-120
Time:                        13:59:02   Log-Likelihood:                -28744.
No. Observations:                3665   AIC:                         5.751e+04
Df Residuals:                    3655   BIC:                         5.757e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          -439.2105    107.175     -4.098

In [54]:
# round(mean_squared_error(np.exp(y_train_log), np.exp(model.fittedvalues), squared=False),3)
round(mean_squared_error(y_train.values, model.fittedvalues, squared=False),3)

616.455

In [55]:
# round(mean_squared_error(np.exp(y_test_log), np.exp(predictions), squared=False),3)
round(mean_squared_error(y_test, predictions, squared=False),3)

676.515

In [56]:
# features
X_iter_ridge = ole_train[['Review_num', "Ratings_num", "Edition_Type", "Genre",
                  "Edition_Month",  "Edition_Year", "Title_n", "Synopsis_n", "Author"]]

X_iter_p_ridge = ole_test[['Review_num', "Ratings_num", "Edition_Type", "Genre",
                  "Edition_Month",  "Edition_Year", "Title_n", "Synopsis_n", "Author"]]

In [57]:
from sklearn.linear_model import Ridge, Lasso

In [58]:
# for i in np.linspace(0,30,100,endpoint=False):
#     rr = Ridge(alpha=i)
#     rr.fit(X_iter_ridge, y_train_log)
#     y_pred = rr.predict(X_iter_p_ridge)
#     print(mean_squared_error(np.exp(y_test_log), np.exp(y_pred), squared=False))
    
# for i in np.linspace(0,30,100,endpoint=False):
#     rr = Ridge(alpha=i)
#     rr.fit(X_iter_ridge, y_train)
#     y_pred = rr.predict(X_iter_p_ridge)
#     print(mean_squared_error(y_test, y_pred, squared=False))

In [59]:
datasetf = pd.read_csv("E:/IIM-A Docs/Internal Activities/Test.csv", header=0, index_col=None, nrows=10000, encoding='iso8859-1')
datasetf.shape

(1000, 9)

In [60]:
datasetf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Title         1000 non-null   object 
 1   Author        1000 non-null   object 
 2   Edition       1000 non-null   object 
 3   Reviews       1000 non-null   object 
 4   Ratings       1000 non-null   object 
 5   Synopsis      1000 non-null   object 
 6   Genre         1000 non-null   object 
 7   BookCategory  1000 non-null   object 
 8   Price         0 non-null      float64
dtypes: float64(1), object(8)
memory usage: 70.4+ KB


In [61]:
datasetf['Review_num'] = datasetf['Reviews'].str.split(' ').str[0].astype(float)

In [62]:
datasetf['Ratings_num'] = datasetf['Ratings'].str.split(' ').str[0]
datasetf['Ratings_num'] = datasetf['Ratings_num'].replace(',','')
datasetf['Ratings_num'] = datasetf['Ratings_num'].apply(lambda x: str(x.replace(',','')))
datasetf['Ratings_num'] = datasetf['Ratings_num'].astype(int)

In [63]:
datasetf['Edition_Type'] = datasetf['Edition'].str.split(' ').str[0]
datasetf['Edition_Type'] = datasetf['Edition_Type'].str.split(',').str[0]

In [64]:
datasetf['Edition_Month'] = datasetf['Edition'].apply(split_month)

In [65]:
datasetf['Edition_Year'] = datasetf['Edition'].apply(split_year)

In [66]:
datasetf['Title_n'] = datasetf['Title'].apply(words_unique)
datasetf['Synopsis_n'] = datasetf['Synopsis'].apply(words_unique)

In [67]:
df_selectcols_f = datasetf[['Review_num', 'Ratings_num', 'Edition_Type',
                        'Genre', 'Edition_Month', 'Edition_Year',
                        'Author', 'Title_n', 'Synopsis_n']]

In [68]:
tmp_mdi_test_f = imputer.transform(df_selectcols_f)

In [69]:
tmp_mdi_test_f.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Review_num     1000 non-null   float64
 1   Ratings_num    1000 non-null   int32  
 2   Edition_Type   1000 non-null   object 
 3   Genre          1000 non-null   object 
 4   Edition_Month  1000 non-null   object 
 5   Edition_Year   1000 non-null   object 
 6   Author         1000 non-null   object 
 7   Title_n        1000 non-null   float64
 8   Synopsis_n     1000 non-null   float64
dtypes: float64(3), int32(1), object(5)
memory usage: 66.5+ KB


In [70]:
eqfdisctr_test_f = eqfdisctr1.transform(tmp_mdi_test_f)

In [71]:
eqfdisctr_test_f1 = eqfdisctr2.transform(eqfdisctr_test_f)

In [72]:
eqfdisctr_test_f1.dtypes

Review_num       object
Ratings_num      object
Edition_Type     object
Genre            object
Edition_Month    object
Edition_Year     object
Author           object
Title_n          object
Synopsis_n       object
dtype: object

In [73]:
rare_test_f = rare_label_encoder.transform(eqfdisctr_test_f1)

In [74]:
ole_test_f = ordered_label_encoder.transform(rare_test_f)

In [75]:
ole_test_f.head()

,Review_num,Ratings_num,Edition_Type,Genre,Edition_Month,Edition_Year,Author,Title_n,Synopsis_n
0,4,14,1,16,3,7,1,0,3
1,7,9,1,16,11,4,1,0,3
2,7,14,3,16,8,4,1,2,1
3,7,14,1,16,4,8,1,0,4
4,7,14,1,3,8,18,1,0,3


In [76]:
X_iter_submission = ole_test_f[['Review_num', "Ratings_num", "Edition_Type", "Genre",
                  "Edition_Month",  "Edition_Year", "Title_n", "Synopsis_n", "Author"]]
X_iter_submission = sm.add_constant(X_iter_submission)
predictions_submission = model.predict(X_iter_submission)

In [77]:
sub_3 = pd.DataFrame(predictions_submission, columns=['Price'])
# sub_3 = pd.DataFrame(np.exp(predictions_submission), columns=['Price'])

In [78]:
# sub_3.to_csv("E:/IIM-A Docs/Internal Activities/sub_3.csv")